In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.distributions import Normal
from torch.utils.data import Subset
from torch.distributions import Categorical, Normal, StudentT
from torch.optim import SGD
from torch.optim.lr_scheduler import PolynomialLR
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as tr
import torchmetrics
from torchmetrics.functional import calibration_error
import math
import matplotlib.pyplot as plt
import random
from collections import deque, OrderedDict
from tqdm import trange
import tqdm
import copy
import typing
from typing import Sequence, Optional, Callable, Tuple, Dict, Union

from data import Data
from Priors import *
from Networks import *
from BayesianNN import BNN_MCMC
from SGLD import SGLD

In [2]:
# Load MNIST dataset with specified transforms
# always use ToTensor() as first transform
# possible transforms: RandomRotation, RandomCrop, GaussianBlur
# avoid Normalize (already done), RandomHorizontalFlip (for MNIST), RandomVerticalFlip (for MNIST)
augmentations = tr.Compose([tr.RandomRotation(15)])

dataset = Data("MNIST", augmentations = None)
train_data, test_data = dataset.get_data()

# Print some information about the dataset
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))


C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:205.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Train data size:  60000
Test data size:  10000


In [7]:
prior = MixedLaplaceUniform()
lol = BNN_MCMC(train_data, network = FullyConnectedNN(), prior=prior, num_epochs = 10, max_size = 10, burn_in = 2, lr = 1e-3, sample_interval = 1)
lol.train()

Training Model


100%|██████████| 10/10 [00:35<00:00,  3.55s/it, acc=1, log_prior_normalized=1.39, loss=1.24e+5, lr=0.000316, nll_loss=6.13]   


In [8]:
acc = lol.test_accuracy(test_data)
ece = lol.test_calibration(test_data)
print("Test accuracy: ", acc)
print("Test ECE: ", ece)

Test accuracy:  tensor(0.9726)
Test ECE:  tensor(0.0118)
